# Lab 6 - Build a Dashboard Application with Plotly Dash

In [1]:
%%capture
!pip3 install pandas dash

In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html as html
from dash import dcc as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

# Read the data into pandas dataframe
import requests
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
resp = requests.get(url)
output = open('spacex_launch_dash.csv', 'wb')
output.write(resp.content)
output.close()

spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Create a dash application
app = dash.Dash(__name__)

app.config.suppress_callback_exceptions = True

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                #==============
                                   html.Div([
                                       # Create an division for adding dropdown helper text for choosing year
                                        html.Div([
                                            html.H2('Launch Site:', style={'margin-right': '2em'})]),
                                        #----
                                       html.Div([dcc.Dropdown(id='site-dropdown', 
                                                     # Update dropdown values using list comphrehension
                                                     options= [{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                                     placeholder="Select a Launch Site",
                                                     value='ALL',
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                            # Place them next to each other using the division style
                                            ], style={'display': 'flex'})]),  
                                
                                #==============

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(
                                    dcc.Graph(id='success-pie-chart')
                                ),
                                #html.Div([ ], id='success-pie-chart', style={'display': 'flex'}),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                html.Div([
                                    dcc.RangeSlider(id='payload-slider',
                                                    min = 0,
                                                    max = 10000,
                                                    step = 1000,
                                                    marks={0: '0',
                                                          2500:'2500',
                                                          5000:'5000',
                                                          7500:'7500',
                                                          10000:'10000'},
                                                    value = [spacex_df['Payload Mass (kg)'].min(), 
                                                             spacex_df['Payload Mass (kg)'].max()]
                                                   )]),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(
                                    dcc.Graph(id='success-payload-scatter-chart'
                                              
                                             ))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
#==============
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
            )
# -Pie Chart
def piechart(site_value):
    if  site_value == 'ALL':
        pie = spacex_df
        pie_fig = px.pie(pie, values = 'class', names='Launch Site', 
                         title='Succesful Launches - All Sites')
        #pie_fig.update_traces(textinfo='value')
    else:
        pie2 = spacex_df[spacex_df['Launch Site'] == site_value]
        d = {'class':[len(pie2[pie2['class']== 1]),len(pie2[pie2['class']== 0])]}
        pie2 = pd.DataFrame(data=d)
        pie_fig = px.pie(pie2, values = 'class', names=pie2.index,
                         title='Succesful Launches for ' + site_value)
        #pie_fig.update_traces(textinfo='value')
    return pie_fig
#==============
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
#==============
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
            )
# -Scatter Chart
def scatterchart(site_value, payload_value):
    if  site_value == 'ALL':
        s_df = spacex_df[(spacex_df['Payload Mass (kg)'] > payload_value[0]) & (spacex_df['Payload Mass (kg)'] < payload_value[1])]
        s_fig = px.scatter(s_df, x='Payload Mass (kg)', y = 'class', color = 'Booster Version Category')
    else:
        s_df = spacex_df[(spacex_df['Payload Mass (kg)'] > payload_value[0]) & (spacex_df['Payload Mass (kg)'] < payload_value[1])]
        s_df = s_df[s_df['Launch Site'] == site_value]
        s_fig = px.scatter(s_df, x='Payload Mass (kg)', y = 'class', color = 'Booster Version Category')
    return s_fig
#==============

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
